In [2]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [3]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import joblib
import seaborn as sns
# min max scaling
from sklearn.preprocessing import MinMaxScaler
# standard scaler
from sklearn.preprocessing import StandardScaler

In [4]:
# list paths in 'data/citylearn_challenge_2022_phase_1/' that start with Build
# and end with .csv
files = [f for f in os.listdir('data/citylearn_challenge_2022_phase_1/') if f.startswith('Build') and f.endswith('.csv')]
files

['Building_4.csv',
 'Building_5.csv',
 'Building_1.csv',
 'Building_2.csv',
 'Building_3.csv']

In [5]:
weather = pd.read_csv('data/citylearn_challenge_2022_phase_1/weather.csv')
weather

,Outdoor Drybulb Temperature [C],Relative Humidity [%],Diffuse Solar Radiation [W/m2],Direct Solar Radiation [W/m2],6h Prediction Outdoor Drybulb Temperature [C],12h Prediction Outdoor Drybulb Temperature [C],24h Prediction Outdoor Drybulb Temperature [C],6h Prediction Relative Humidity [%],12h Prediction Relative Humidity [%],24h Prediction Relative Humidity [%],6h Prediction Diffuse Solar Radiation [W/m2],12h Prediction Diffuse Solar Radiation [W/m2],24h Prediction Diffuse Solar Radiation [W/m2],6h Prediction Direct Solar Radiation [W/m2],12h Prediction Direct Solar Radiation [W/m2],24h Prediction Direct Solar Radiation [W/m2]
0,20.0,84.0,0.0,0.0,18.3,22.8,20.0,81.0,68.0,81.0,25.0,964.0,0.0,100.0,815.0,0.0
1,20.1,79.0,0.0,0.0,19.4,22.8,19.4,79.0,71.0,87.0,201.0,966.0,0.0,444.0,747.0,0.0
2,19.7,78.0,0.0,0.0,21.1,22.2,19.4,73.0,73.0,87.0,420.0,683.0,0.0,592.0,291.0,0.0
3,19.3,78.0,0.0,0.0,22.2,22.8,19.4,71.0,71.0,90.0,554.0,522.0,0.0,491.0,153.0,0.0
4,18.9,78.0,0.0,0.0,21.7,22.2,18.9,73.0,71.0,90.0,778.0,444.0,0.0,734.0,174.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,20.6,84.0,26.0,130.0,20.1,19.4,20.6,79.0,79.0,73.0,0.0,201.0,27.0,0.0,444.0,143.0
8756,21.1,81.0,0.0,0.0,19.7,21.1,20.0,78.0,73.0,76.0,0.0,420.0,0.0,0.0,592.0,0.0
8757,21.7,79.0,0.0,0.0,19.3,22.2,20.6,78.0,71.0,70.0,0.0,554.0,0.0,0.0,491.0,0.0
8758,21.3,76.0,0.0,0.0,18.9,21.7,20.6,78.0,73.0,73.0,0.0,778.0,0.0,0.0,734.0,0.0


In [6]:
weather.columns

Index(['Outdoor Drybulb Temperature [C]', 'Relative Humidity [%]',
       'Diffuse Solar Radiation [W/m2]', 'Direct Solar Radiation [W/m2]',
       '6h Prediction Outdoor Drybulb Temperature [C]',
       '12h Prediction Outdoor Drybulb Temperature [C]',
       '24h Prediction Outdoor Drybulb Temperature [C]',
       '6h Prediction Relative Humidity [%]',
       '12h Prediction Relative Humidity [%]',
       '24h Prediction Relative Humidity [%]',
       '6h Prediction Diffuse Solar Radiation [W/m2]',
       '12h Prediction Diffuse Solar Radiation [W/m2]',
       '24h Prediction Diffuse Solar Radiation [W/m2]',
       '6h Prediction Direct Solar Radiation [W/m2]',
       '12h Prediction Direct Solar Radiation [W/m2]',
       '24h Prediction Direct Solar Radiation [W/m2]'],
      dtype='object')

In [7]:
def add_lag_feature(df, window=3, lag_cols=["net_target"]):
    rolled = df[lag_cols].rolling(window=window, min_periods=0, center=False)
    lag_mean = rolled.mean().reset_index().astype(np.float16)
    lag_max = rolled.quantile(0.95).reset_index().astype(np.float16)
    lag_min = rolled.quantile(0.05).reset_index().astype(np.float16)
    lag_std = rolled.std().reset_index().astype(np.float16)
    
    for col in lag_cols:
        df[f"{col}_mean_lag{window}"] = lag_mean[col]
        df[f"{col}_max_lag{window}"] = lag_max[col]
        df[f"{col}_min_lag{window}"] = lag_min[col]
        df[f"{col}_std_lag{window}"] = lag_std[col]

In [83]:
# loop through csv files and extract consumption and generation data
all_buildings = pd.DataFrame()
for i, file in enumerate(files):
    df = pd.read_csv('data/citylearn_challenge_2022_phase_1/' + file)
    filename = file.split('.')[0].split('_')[1]
    df = pd.concat([df, weather], axis=1)
    df['building'] = i
    df['day_year'] = df.index + 23
    df['dayinyear'] = df['day_year'] // 24 
    df['Hour'] = df['Hour'] - 1
    # create a timestamp column from the dayinyear and hour from origin 23:00 31/07/2021
    df['timestamp'] = pd.to_datetime(df['dayinyear'], unit='D', origin='2021-07-31 23:00') + pd.to_timedelta(df['Hour'], unit='h')
    df['number'] = filename
    # weekday hour interaction feature
    df['weekday_hour'] =  df['Hour'] * 10 + df['Day Type']
    df["hour_x"] = np.cos(2*np.pi* df["Hour"] / 24)
    df["hour_y"] = np.sin(2*np.pi* df["Hour"] / 24)
    
    df["month_x"] = np.cos(2*np.pi* df["Month"] / (12))
    df["month_y"] = np.sin(2*np.pi*df["Month"] / (12))
    
    df["weekday_x"] = np.cos(2 * np.pi*df["Day Type"] / (7))
    df["weekday_y"] = np.sin(2 * np.pi*df["Day Type"] / (7))
    # target one step agead
    if filename == '4':
        df['net'] = df['Equipment Electric Power [kWh]'] - (df['Solar Generation [W/kW]'] * 5)/1000
    else:
        df['net'] = df['Equipment Electric Power [kWh]'] - (df['Solar Generation [W/kW]'] * 4)/1000
    df['net_target'] = df['net']
    df['net+1'] = df['net'].shift(-1)
    df['cons+1'] = df['Equipment Electric Power [kWh]'].shift(-1)
    df['gen+1'] = df['Solar Generation [W/kW]'].shift(-1)
    # rename the equipment electric power column to cons_target
    df = df.rename(columns={'Equipment Electric Power [kWh]': 'cons_target-1'})
    df.drop(['Daylight Savings Status','DHW Heating [kWh]',
       'Cooling Load [kWh]', 'Heating Load [kWh]'], axis=1, inplace=True)

    scaler = MinMaxScaler()
    # Fit and apply minmax scaler
    df['net_target-1'] = scaler.fit_transform(df['net_target'].values.reshape(-1, 1))
    df['net_target'] = df['net_target-1'].shift(-1)
    # save scaler
    scaler_name = 'scalers/scaler_net_' + filename + '.save'
    joblib.dump(scaler, scaler_name)
    # Fit and apply minmax scaler
    df['cons_target-1'] = scaler.fit_transform(df['cons_target-1'].values.reshape(-1, 1))
    # save scaler
    scaler_name = 'scalers/scaler_cons_' + filename + '.save'
    joblib.dump(scaler, scaler_name)

    #if the column name contains 'Radiation' or 'radiation', log transform
    for col in df.columns:
        if 'Radiation' in col or 'radiation' in col:
            df[col] = np.log1p(df[col])

    # add lag features
    add_lag_feature(df, window=3, lag_cols=["net_target-1"])
    df['cons_target'] = df['cons_target-1'].shift(-1)
    df['gen_target'] = df['Solar Generation [W/kW]'].shift(-1)
    df['cons_target-2'] = df['cons_target-1'].shift(1)
    df['gen_target-2'] = df['Solar Generation [W/kW]'].shift(1)
    df['diffuse_solar_radiation+1'] = df['Diffuse Solar Radiation [W/m2]'].shift(-1)
    df['drybulb_temp+1'] = df['Outdoor Drybulb Temperature [C]'].shift(-1)
    df['relative_humidity+1'] = df['Relative Humidity [%]'].shift(-1)
    df['net_target-23'] = df['net_target'].shift(23)
    #add_lag_feature(df, window=3, lag_cols=["diffuse_solar_radiation+1", 'drybulb_temp+1', 'relative_humidity+1'])

    # remove the columns with Prediction in string
    df = df[df.columns.drop(list(df.filter(regex='Prediction')))]
    all_buildings = pd.concat([all_buildings, df], axis=0)
    print(len(all_buildings))
# drop columns with more than 1000 missing values
all_buildings = all_buildings[all_buildings.columns[all_buildings.isnull().mean() < 0.8]]
all_buildings

8760
17520
26280
35040
43800


,Month,Hour,Day Type,cons_target-1,Solar Generation [W/kW],Outdoor Drybulb Temperature [C],Relative Humidity [%],Diffuse Solar Radiation [W/m2],Direct Solar Radiation [W/m2],building,...,net_target-1_min_lag3,net_target-1_std_lag3,cons_target,gen_target,cons_target-2,gen_target-2,diffuse_solar_radiation+1,drybulb_temp+1,relative_humidity+1,net_target-23
0,7.0,23.0,7.0,0.417676,0.000000,20.0,84.0,0.000000,0.000000,0,...,0.743164,NaN,0.285671,0.0,NaN,NaN,0.0,20.1,79.0,NaN
1,8.0,0.0,1.0,0.285671,0.000000,20.1,79.0,0.000000,0.000000,0,...,0.636719,0.079224,0.070495,0.0,0.417676,0.000000,0.0,19.7,78.0,NaN
2,8.0,1.0,1.0,0.070495,0.000000,19.7,78.0,0.000000,0.000000,0,...,0.467041,0.148682,0.067890,0.0,0.285671,0.000000,0.0,19.3,78.0,NaN
3,8.0,2.0,1.0,0.067890,0.000000,19.3,78.0,0.000000,0.000000,0,...,0.446777,0.106079,0.066080,0.0,0.070495,0.000000,0.0,18.9,78.0,NaN
4,8.0,3.0,1.0,0.066080,0.000000,18.9,78.0,0.000000,0.000000,0,...,0.445068,0.001884,0.069024,0.0,0.067890,0.000000,0.0,18.3,81.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,7.0,18.0,1.0,0.241327,13.670833,20.6,84.0,3.295837,4.875197,4,...,0.451416,0.073242,0.176317,0.0,0.257741,210.758333,0.0,21.1,81.0,0.533175
8756,7.0,19.0,1.0,0.176317,0.000000,21.1,81.0,0.000000,0.000000,4,...,0.507324,0.044128,0.190595,0.0,0.241327,13.670833,0.0,21.7,79.0,0.503666
8757,7.0,20.0,1.0,0.190595,0.000000,21.7,79.0,0.000000,0.000000,4,...,0.548340,0.022797,0.181485,0.0,0.176317,0.000000,0.0,21.3,76.0,0.499451
8758,7.0,21.0,1.0,0.181485,0.000000,21.3,76.0,0.000000,0.000000,4,...,0.547363,0.005657,0.123995,0.0,0.190595,0.000000,0.0,20.9,76.0,0.499823


In [91]:
all_buildings.dropna(inplace=True)
all_buildings.sort_values(['number', 'day_year'], inplace=True)
all_buildings.reset_index(drop=True, inplace=True)
# remove rows with dayinyear 1 and 365
all_buildings = all_buildings[all_buildings['dayinyear'] != 1]
all_buildings = all_buildings[all_buildings['dayinyear'] != 365]
# save building 4 to test and 0,1,2,3 to train
all_buildings[all_buildings['building'] == 4].to_csv('data/extra_test.csv')
all_buildings[all_buildings['building'] != 4].to_csv('data/extra_train.csv')

/Users/evgenygenov/miniforge3/envs/citylearn/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [89]:
all_buildings[all_buildings['building'] != 4][['Month', 'Hour', 'Day Type', 'day_year','dayinyear','timestamp', 'number']]

,Month,Hour,Day Type,day_year,dayinyear,timestamp,number
0,8.0,22.0,1.0,46,1,2021-08-01 22:00:00,1
1,8.0,23.0,1.0,47,1,2021-08-01 23:00:00,1
2,8.0,0.0,2.0,48,2,2021-08-02 00:00:00,1
3,8.0,1.0,2.0,49,2,2021-08-02 01:00:00,1
4,8.0,2.0,2.0,50,2,2021-08-02 02:00:00,1
...,...,...,...,...,...,...,...
43675,7.0,17.0,1.0,8777,365,2022-07-31 17:00:00,5
43676,7.0,18.0,1.0,8778,365,2022-07-31 18:00:00,5
43677,7.0,19.0,1.0,8779,365,2022-07-31 19:00:00,5
43678,7.0,20.0,1.0,8780,365,2022-07-31 20:00:00,5
